**PDF Preprocessing (Extract & Clean Text)**
* Convert PDFs to text
  - Use PyMuPDF or pdfplumber to extract text.
* Clean & Structure the Text
  - Remove headers, footers, and page numbers.
  - Split text into chapters or meaningful chunks (~512-2048 tokens each).
  - Convert text into structured format (JSON, Markdown, or plain text).
* Tokenize for Embedding Model
  - Use a tokenizer (like sentence-transformers or tiktoken) to ensure cleanhunking for embeddings.

**Storing Data in a Vector Database (ChromaDB)**
* Select an Embedding Model
sentence-transformers (local, e.g., all-MiniLM-L6-v2)
* Generate & Store Embeddings
  - Convert text chunks into dense vectors.
  - Store in ChromaDB.
* Metadata Storage
   - Keep chapter titles, book names, and page numbers for context retrieval.

**Structured Prompting & Querying the Database**
* User Input: The model receives a structured prompt.
* Search Vector Database:
  - Convert input into an embedding.
   - Retrieve the top-k most relevant passages.
* Construct a Contextual Prompt:
  - Use retrieved passages to create a prompt with in-context learning (ICL).

**Generative Model (LLaMA 2) for Novel Writing**
* Model Selection:
  - LLaMA 2 fine-tuned version for stylistic improvements).
* Generation Process:
  - Feed structured prompt into LLaMA 2.
  - Use temperature & top-k sampling for creative output.
* Iterative Refinement:
  - Generate chapters iteratively rather than the full novel at once.

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

####**Tokenization**

In [ ]:
pip install pymupdf pdfplumber tiktoken nltk

In [ ]:
import os
from sentence_transformers import SentenceTransformer

# Define input/output directories
input_folder = "/content/drive/My Drive/Portfolio/Novel_RAG_Project/Data/Chunked_Texts"
output_folder = "/content/drive/My Drive/Portfolio/Novel_RAG_Project/Data/tokenized_BettyNeelsDataset"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load embedding model (which includes a tokenizer)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Process each chunked text file
for filename in os.listdir(input_folder):
    if filename.endswith("_chunked.txt"):  # Ensure we're processing chunked files only
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, filename.replace("_chunked.txt", "_tokenized.txt"))

        with open(input_path, 'r', encoding='utf-8') as file:
            text = file.read()

        # Split chunks based on delimiter
        chunks = text.split("\n\n### CHUNK ")

        tokenized_chunks = []
        for chunk_id, chunk in enumerate(chunks):
            if chunk.strip():
                tokens = model.tokenizer.tokenize(chunk)  # Tokenize text
                tokenized_text = " ".join(tokens)  # Convert list of tokens back to text
                tokenized_chunks.append(f"### CHUNK {chunk_id+1} ###\n{tokenized_text}")

        # Join tokenized chunks with blank lines and save to new file
        with open(output_path, 'w', encoding='utf-8') as output_file:
            output_file.write("\n\n".join(tokenized_chunks))

        print(f"Tokenized and saved: {filename} → {output_path}")

print("All Files Tokenized.")

####**Load LLaMA 2**

In [ ]:
# Install dependencies
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub chromadb sentence-transformers

In [ ]:
import os
import torch
import chromadb
from huggingface_hub import hf_hub_download
from llama_cpp import Llama


# Define input folder (Chunked text files)
input_folder = "/content/drive/My Drive/Portfolio/Novel_RAG_Project/Data/Chunked_Texts"

# Load embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize ChromaDB client & collection
client = chromadb.PersistentClient(path="/content/chroma_db")  # Persistent storage
collection = client.get_or_create_collection(name="betty_neels_books")

# Load LLaMA model
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

# Download model
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# Initialize LLaMA model
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4,  # Increase CPU cores if available
    n_batch=512,  # Adjust for GPU VRAM
    n_gpu_layers=32  # Optimize based on available GPU memory
)

print("LLaMA model loaded successfully.")

####**Extract and Store Embeddings, Model Prompting**

In [ ]:
# Function to retrieve relevant chunks from ChromaDB
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = embedding_model.encode(query)  # Convert query into vector

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k  # Retrieve top-k most relevant passages
    )

    retrieved_chunks = []
    if "documents" in results and results["documents"]:
        for i in range(len(results["documents"][0])):  # Loop through top-k results
            chunk_text = results["documents"][0][i]
            metadata = results["metadatas"][0][i]  # Metadata for context
            retrieved_chunks.append((chunk_text, metadata))

    return retrieved_chunks

# Function to construct the structured prompt
def construct_prompt(query, retrieved_chunks):
    prompt = "You are a writer skilled in the style of Betty Neels. Use the provided passages as inspiration to generate a new novel passage.\n\n"
    prompt += f"### User Prompt: {query}\n\n"

    for i, (chunk, metadata) in enumerate(retrieved_chunks):
        prompt += f"### Retrieved Passage {i+1} from {metadata['book_name']}, {metadata['chapter_title']}:\n"
        prompt += chunk + "\n\n"

    prompt += "### Now generate a continuation based on the style and content above."
    return prompt

# Function to generate text using LLaMA 2
def generate_text(prompt, max_tokens=500):
    output = lcpp_llm(
        prompt,
        max_tokens=max_tokens,  # Limit response length
        temperature=0.7,  # Adjust creativity
        top_p=0.9
    )
    return output["choices"][0]["text"]

# RAG query
user_query = "Write a chapter in Betty Neels’ style about a nurse meeting a wealthy doctor in Holland."

retrieved_chunks = retrieve_relevant_chunks(user_query, top_k=3)  # Retrieve relevant passages
structured_prompt = construct_prompt(user_query, retrieved_chunks)  # Build prompt
generated_text = generate_text(structured_prompt)  # Generate output

# Display the output
print("\n Generated Text:\n")
print(generated_text)

**OUTPUT**

Generated Text:

Passage 1:

"The sun was setting over the windmills of Holland, casting a golden glow over the landscape. Nurse Emily walked down the cobblestone street, her starched cap and crisp white apron fluttering in the breeze. She had just finished her shift at the local hospital and was looking forward to a well-deserved rest. As she passed by the grand manor house on the hill, she noticed a handsome doctor standing in the doorway, watching her with piercing blue eyes."

Passage 2:

"Dr. van der Meer was not only one of the most eligible bachelors in Holland, but also one of its most skilled physicians. He had built his reputation on his exceptional bedside manner and his ability to heal even the most stubborn cases. But as he gazed at Nurse Emily, he felt a stirring in his chest that he had never experienced before. She was not like the other nurses, with their curly hair and rosy cheeks. No, Emily was different - her dark hair was pulled back into a neat cap, revealing her high forehead and piercing brown eyes. He found himself drawn to her intelligence and self-assurance."

Continuation:

As Nurse Emily approached the manor house, she felt a sense of unease wash over her. She had heard whispers about Dr. van der Meer's reputation as a ladies' man, and she didn't want to be just another notch on his belt. But as she entered the grand foyer, she was struck by the warmth in his eyes. He greeted her with a bow, his deep voice sending shivers down her spine.

"Welcome, Nurse Emily," he said, offering his hand. "I have been expecting you. I have a special task for you, one that requires not only your nursing skills but also your discretion

In [ ]:
# Function to retrieve relevant chunks from ChromaDB
def retrieve_relevant_chunks(query, top_k=3):
    query_embedding = embedding_model.encode(query)  # Convert query into vector

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k  # Retrieve top-k most relevant passages
    )

    retrieved_chunks = []
    if "documents" in results and results["documents"]:
        for i in range(len(results["documents"][0])):  # Loop through top-k results
            chunk_text = results["documents"][0][i]
            metadata = results["metadatas"][0][i]  # Metadata for context
            retrieved_chunks.append((chunk_text, metadata))

    return retrieved_chunks

# 🔹 Function to construct the structured prompt
def construct_prompt(query, retrieved_chunks):
    prompt = "You are a writer skilled in the style of Betty Neels. Use the provided passages as inspiration to generate a new novel passage.\n\n"
    prompt += f"### User Prompt: {query}\n\n"

    for i, (chunk, metadata) in enumerate(retrieved_chunks):
        prompt += f"### Retrieved Passage {i+1} from {metadata['book_name']}, {metadata['chapter_title']}:\n"
        prompt += chunk + "\n\n"

    prompt += "### Now generate a continuation based on the style and content above."
    return prompt

# 🔹 Function to generate text using LLaMA 2
def generate_text(prompt, max_tokens=500):
    output = lcpp_llm(
        prompt,
        max_tokens=max_tokens,  # Limit response length
        temperature=0.7,  # Adjust creativity
        top_p=0.9
    )
    return output["choices"][0]["text"]

# 🔹 Example RAG query
user_query = "Write a chapter in Betty Neels’ style about a nurse from London, going to Scotland to work with a doctor."

retrieved_chunks = retrieve_relevant_chunks(user_query, top_k=3)  # Retrieve relevant passages
structured_prompt = construct_prompt(user_query, retrieved_chunks)  # Build prompt
generated_text = generate_text(structured_prompt)  # Generate output

# 🔹 Display the output
print("\n📝 Generated Text:\n")
print(generated_text)

**OUTPUT:**

Passage 1:

"The train chugged out of King's Cross station, carrying Emily, a young nurse from London, northwards to Scotland. She had accepted a position at a small hospital in the picturesque town of Drumnadrochit, nestled amongst the rugged hills and lochs of the Scottish Highlands. The journey was long, but Emily was filled with excitement at the prospect of starting her new life in this remote and beautiful place."

Passage 2:

"As the train rumbled on, Emily couldn't help but feel a sense of nervous anticipation. She had never been to Scotland before, and the thought of working with Dr. MacTavish, the hospital's esteemed doctor, made her heart race. She had heard tales of his strict nature and high expectations, but she was determined to prove herself as a capable and dedicated nurse. The sound of the wind rustling through the trees outside her window only added to her sense of adventure."

Passage 3:

"When the train finally arrived in Drumnadrochit, Emily was struck by the breathtaking beauty of the town. The hospital sat perched on a hill, its white walls and slate roof gleaming in the fading light of day. As she made her way to her new home, Emily couldn't help but feel a sense of wonder at the rugged landscape that surrounded her. She had never felt so alive, so ready for whatever challenges lay ahead."

Continuation:

As Emily settled into her small cottage on the hospital grounds, she couldn't help but feel a sense of trepidation about her new role. The other nurses seemed friendly enough, but she knew that Dr. MacTavish was notoriously difficult to work with. Still, she was determined to prove herself and make the most of this exciting new opportunity.

The next morning, Emily reported for duty bright